In [271]:
from pathlib import Path 
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd
from collections import OrderedDict
import sys
import os
import seaborn as sns
import researchpy as rp
import statsmodels.formula.api as smf
import scipy.stats as stats
from sklearn.preprocessing import StandardScaler

#sys.path.append('/Users/alina/Desktop/MIT/code/ADHD/MTA/helper')
from helper import rr, prep, var_dict

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [272]:
#%reload_ext autoreload

In [273]:
data_root = '/Volumes/Samsung_T5/MIT/mta'
derived_data = '/Volumes/Samsung_T5/MIT/mta/output/derived_data'
#os.listdir(data_root)


In [274]:
baseline_var = ['src_subject_id', 'interview_date', 'interview_age', 'sex', 'site', 'days_baseline']
dtypes_baseline = { 'src_subject_id' : 'str',
                    'interview_date': 'str' , 
                    'interview_age' : 'int64' ,
                    'sex' : 'str', 
                    'site' : 'int64' ,
                    'days_baseline':  'int64',
                    'version_form': 'str'}
version_form = ['version_form']

qsts = ['snap', 'ssrs',  'pc', 'wechsler'] #masc to many missing data 

In [327]:
#outcome variablles 
snap_vars = ['snainatx', 'snahypax', 'snaoddx'] #inattention_mean, hyperactie mean
ssrs_vars = ['sspintx', 'ssptossx']# social skills mean, internalizing mean 
#masc_vars = ['masc_masctotalt']
pc_vars = ['pcrcpax', 'pcrcprx'] # power assertion, personal closeness
wechsler_vars = ['w1readb','w2math','w3spell' ]
outcomes_dict  = {'snap' : snap_vars, 'ssrs' : ssrs_vars,  'pc': pc_vars, 'wechsler': wechsler_vars}


In [276]:
interaction_predictors = ['days_baseline', 'site', 'trtname'] #time, site, treatment group

# mediator variables
comorb_mediators  = ['cdorodd' , 'pso', 'psoi', 'pag', 'pagi', 'pga', 'pgai' ,'psa'] #ODD/CD or anx excluding specific phobia 
services_mediators =  ['demo61'] #reciept of public assistance 
prev_med_mediators = ['hi_24'] #medication intake prior to study 

#moderator variables 
accept_moderator = ['d2dresp'] # initail acceptance of treatment 

In [328]:
# load files, drop rows if missing date, drop duplicates 

snap_file = 'snap01.txt'
ssrs_file = 'ssrs01.txt'
#masc_file = 'masc_p01.txt'
parent_child_file = 'pcrc01.txt'
wechsler_file = 'wiat_iiip201.txt'
treat_group = 'treatment_groups.csv'

snap_ = pd.read_csv(Path(data_root, snap_file), delimiter="\t", usecols=np.concatenate((baseline_var, snap_vars, version_form)), skiprows=[1] , parse_dates=['days_baseline']).dropna(subset='days_baseline').drop_duplicates()
ssrs_ = pd.read_csv(Path(data_root, ssrs_file), delimiter="\t", usecols=np.concatenate((baseline_var, ssrs_vars, version_form)), skiprows=[1] , parse_dates=['days_baseline']).dropna(subset='days_baseline').drop_duplicates()
#masc_ = pd.read_csv(Path(data_root, masc_file), delimiter="\t", usecols=np.concatenate((baseline_var, masc_vars, version_form)), skiprows=[1] , parse_dates=['days_baseline']).dropna(subset='days_baseline').drop_duplicates()
pc_ = pd.read_csv(Path(data_root, parent_child_file), delimiter="\t", usecols=np.concatenate((baseline_var, pc_vars, version_form)), skiprows=[1] , parse_dates=['days_baseline']).dropna(subset='days_baseline').drop_duplicates()
wechsler_ = pd.read_csv(Path(data_root, wechsler_file), delimiter="\t", usecols=np.concatenate((baseline_var, wechsler_vars)), skiprows=[1] , parse_dates=['days_baseline']).dropna(subset='days_baseline').drop_duplicates()
treat_group = pd.read_csv(Path(derived_data, treat_group))
orig_rr_14 = pd.read_csv(Path(derived_data, 'original_rr_results_14.csv'))

/var/folders/hf/frc4nxb532j7gqr3_36l6kmh0000gn/T/ipykernel_39023/1258586597.py:10: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  snap_ = pd.read_csv(Path(data_root, snap_file), delimiter="\t", usecols=np.concatenate((baseline_var, snap_vars, version_form)), skiprows=[1] , parse_dates=['days_baseline']).dropna(subset='days_baseline').drop_duplicates()
/var/folders/hf/frc4nxb532j7gqr3_36l6kmh0000gn/T/ipykernel_39023/1258586597.py:11: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  ssrs_ = pd.read_csv(Path(data_root, ssrs_file), delimiter="\t", usecols=np.concatenate((baseline_var, ssrs_vars, version_form)), skiprows=[1] , parse_dates=['days_baseline']).dropna(subset='days_baseline').drop_duplicates()
/var/folders/h

In [329]:
# merge with treatment group info convert colum data to appropriate dtypes 

snap = prep.set_baseline_dtypes(pd.merge(snap_, treat_group, how='inner', on = 'src_subject_id')).dropna()#.table with relevant snap vales, rater, and treatment group 
ssrs = prep.set_baseline_dtypes(pd.merge(ssrs_, treat_group, how='inner', on = 'src_subject_id').dropna())#.dropna() #table with relevant snap vales, rater, and treatment group 
#masc = prep.set_baseline_dtypes(pd.merge(masc_, treat_group, how='inner', on = 'src_subject_id')).dropna()#.dropna() #table with relevant snap vales, rater, and treatment group 
pc = prep.set_baseline_dtypes(pd.merge(pc_, treat_group, how='inner', on = 'src_subject_id').dropna())#.dropna() #table with relevant snap vales, rater, and treatment group 
wechsler = prep.set_baseline_dtypes(pd.merge(wechsler_, treat_group, how='inner', on = 'src_subject_id')).dropna()#.dropna() #table with relevant snap vales, rater, and treatment group 

print(snap_.shape, ssrs.shape, pc.shape, wechsler.shape)

Success
Success
Success
Success
(14540, 10) (10735, 10) (8627, 10) (4571, 10)


In [279]:
ssrs.loc[ssrs['version_form'].str.startswith('Teacher'), 'version_form'] = 'Teacher'
ssrs.loc[ssrs['version_form'].str.startswith('Parent'), 'version_form'] = 'Parent'

In [280]:
comorb_file = 'diagpsx01.txt' # contains odd/cd and anx comorbid diagnoses 
demog_file = 'demgr01.txt' # contains recieot of public assistance 
health_qst_file  = 'health01.txt' # contains previous medication 
init_sat_file = 'debrief01.txt' # contains rating of initial acceptance of treatment goup

comorb = prep.set_baseline_dtypes(pd.read_csv(Path(data_root, comorb_file), delimiter= '\t', usecols = np.concatenate((baseline_var, comorb_mediators)), skiprows=[1] , parse_dates=['days_baseline']).dropna(subset='days_baseline').drop_duplicates())
demog = prep.set_baseline_dtypes(pd.read_csv(Path(data_root, demog_file), delimiter= '\t', usecols  =np.concatenate((baseline_var, services_mediators)), skiprows=[1] , parse_dates=['days_baseline']).dropna(subset='days_baseline').drop_duplicates())
health_qst = prep.set_baseline_dtypes(pd.read_csv(Path(data_root, health_qst_file), delimiter= '\t', usecols=np.concatenate((baseline_var, prev_med_mediators)), skiprows=[1] , parse_dates=['days_baseline']).dropna(subset='days_baseline').drop_duplicates())
init_sat_file = prep.set_baseline_dtypes(pd.read_csv(Path(data_root, init_sat_file), delimiter= '\t', usecols=np.concatenate((baseline_var,  accept_moderator)), skiprows=[1] , parse_dates=['days_baseline']).dropna(subset='days_baseline').drop_duplicates())



Success
Success
Success
Success


/var/folders/hf/frc4nxb532j7gqr3_36l6kmh0000gn/T/ipykernel_39023/3879433410.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  comorb = prep.set_baseline_dtypes(pd.read_csv(Path(data_root, comorb_file), delimiter= '\t', usecols = np.concatenate((baseline_var, comorb_mediators)), skiprows=[1] , parse_dates=['days_baseline']).dropna(subset='days_baseline').drop_duplicates())
/var/folders/hf/frc4nxb532j7gqr3_36l6kmh0000gn/T/ipykernel_39023/3879433410.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  demog = prep.set_baseline_dtypes(pd.read_csv(Path(data_root, demog_file), delimiter= '\t', usecols  =np.concatenate((baseline_var, services_mediators)), skiprows=[1] , parse_dates=['days_baseline']).dropna(subset='d

In [281]:
snap_split_dict = prep.split_data_from_timepoints(snap)
ssrs_split_dict = prep.split_data_from_timepoints(ssrs)
pc_split_dict = prep.split_data_from_timepoints(pc)
wechsler_split_dict = prep.split_data_from_timepoints(wechsler)
data = {'snap' : snap_split_dict, 'ssrs': ssrs_split_dict, 'pc': pc_split_dict, 'wechsler': wechsler_split_dict}

In [282]:
pc_split_dict['14']['days_baseline'].value_counts()

days_baseline
0      2880
419      36
196      31
392      30
448      30
       ... 
49        1
63        1
308       1
307       1
327       1
Name: count, Length: 437, dtype: int64

In [283]:
data['pc']['14']['days_baseline'] == pc_split_dict['14']['days_baseline']

0       True
1       True
2       True
3       True
4       True
        ... 
8665    True
8666    True
8667    True
8668    True
8669    True
Name: days_baseline, Length: 6985, dtype: bool

In [284]:
data['pc']['14']['days_baseline'].value_counts()

days_baseline
0      2880
419      36
196      31
392      30
448      30
       ... 
49        1
63        1
308       1
307       1
327       1
Name: count, Length: 437, dtype: int64

In [285]:
gen_interact_formula = 'C(trtname, Treatment(reference = "L")) * days_baseline * C(site)' # reapeat with log days 
formulas =  [[' ~ '.join((var, gen_interact_formula)) for var in values] for values in outcomes_dict.values()]
formulas_dict = dict(zip(outcomes_dict.keys(), formulas))
formulas_dict

{'snap': ['snainatx ~ C(trtname, Treatment(reference = "L")) * days_baseline * C(site)',
  'snahypax ~ C(trtname, Treatment(reference = "L")) * days_baseline * C(site)',
  'snaoddx ~ C(trtname, Treatment(reference = "L")) * days_baseline * C(site)'],
 'ssrs': ['sspintx ~ C(trtname, Treatment(reference = "L")) * days_baseline * C(site)',
  'ssptossx ~ C(trtname, Treatment(reference = "L")) * days_baseline * C(site)'],
 'pc': ['pcrcpax ~ C(trtname, Treatment(reference = "L")) * days_baseline * C(site)',
  'pcrcprx ~ C(trtname, Treatment(reference = "L")) * days_baseline * C(site)'],
 'wechsler': ['w1readb ~ C(trtname, Treatment(reference = "L")) * days_baseline * C(site)',
  'w2math ~ C(trtname, Treatment(reference = "L")) * days_baseline * C(site)',
  'w3spell ~ C(trtname, Treatment(reference = "L")) * days_baseline * C(site)']}

In [286]:

groups = 'src_subject_id'
alpha = 0.05
hyps_interactions = var_dict.get_hyps_interactions()

In [287]:
hyps_interactions

{'time_treat': 'C(trtname, Treatment(reference="L"))[T.M]:days_baseline = C(trtname, Treatment(reference="L"))[T.P]:days_baseline  = C(trtname, Treatment(reference="L"))[T.C]:days_baseline = 0',
 'site_treat': 'C(trtname, Treatment(reference="L"))[T.M]:C(site)[T.2] = C(trtname, Treatment(reference="L"))[T.P]:C(site)[T.2] = C(trtname, Treatment(reference="L"))[T.C]:C(site)[T.2] = C(trtname, Treatment(reference="L"))[T.M]:C(site)[T.3] = C(trtname, Treatment(reference="L"))[T.P]:C(site)[T.3] = C(trtname, Treatment(reference="L"))[T.C]:C(site)[T.3] = C(trtname, Treatment(reference="L"))[T.M]:C(site)[T.4] = C(trtname, Treatment(reference="L"))[T.P]:C(site)[T.4] = C(trtname, Treatment(reference="L"))[T.C]:C(site)[T.4] = C(trtname, Treatment(reference="L"))[T.M]:C(site)[T.5] = C(trtname, Treatment(reference="L"))[T.P]:C(site)[T.5] = C(trtname, Treatment(reference="L"))[T.C]:C(site)[T.5] = C(trtname, Treatment(reference="L"))[T.M]:C(site)[T.6] = C(trtname, Treatment(reference="L"))[T.P]:C(site

In [288]:
# result, summ, hsmm = rr.get_RR_stats(formula, data, groups=groups, alpha= alpha)
# rr.f_test_interactions(result, hyps_interactions, alpha)

In [308]:
raters = ['Teacher', 'Parent']

In [309]:
result_snap = [[ smf.mixedlm(formulas_dict['snap'][i], data['snap']['14'][data['snap']['14']['version_form'] == rater], groups = groups).fit() for rater in raters] for i in range(len(snap_vars))]

In [310]:
result_ssrs = [[ smf.mixedlm(formulas_dict['ssrs'][i], data['ssrs']['14'][data['ssrs']['14']['version_form'] == rater], groups = groups).fit() for rater in raters] for i in range(len(ssrs_vars))]

In [311]:
#test = data['pc']['14'][data['pc']['14']['version_form'] == 'Parent'].copy()

In [312]:
#scaler = StandardScaler()

In [313]:
#test['days_baseline'] = scaler.fit_transform(np.array(test['days_baseline']).reshape(-1, 1))

In [314]:
#test_pc_formula = 'pcrcpax ~ C(trtname, Treatment(reference = "L"))* days_baseline  '

In [315]:
#result = smf.mixedlm(formulas_dict['pc'][0], data['pc']['14'][data['pc']['14']['version_form'] == 'Parent'], groups = groups).fit() 

In [316]:
#result.summary()

In [317]:
# from statsmodels.stats.outliers_influence import variance_inflation_factor
# X = result.model.exog
# vif = pd.DataFrame()
# vif["VIF Factor"] = [variance_inflation_factor(X, i) for i in range(X.shape[1])]
# vif["features"] = result.model.exog_names
# print(vif)

In [318]:
#result_pc2 =  smf.mixedlm(formulas_dict['pc'][1], data['pc']['14'][data['pc']['14']['version_form'] == 'Parent'], groups = groups).fit()  

In [319]:
# result_wechlser = [ smf.mixedlm(formulas_dict['wechsler'][i], data['wechsler']['14'], groups = groups).fit()  for i in range(len(wechsler_vars))]

In [320]:
## worked for snap, ssrs, wechsler

In [321]:
hyps_interactions = var_dict.get_hyps_interactions()
hyps_interactions

{'time_treat': 'C(trtname, Treatment(reference="L"))[T.M]:days_baseline = C(trtname, Treatment(reference="L"))[T.P]:days_baseline  = C(trtname, Treatment(reference="L"))[T.C]:days_baseline = 0',
 'site_treat': 'C(trtname, Treatment(reference="L"))[T.M]:C(site)[T.2] = C(trtname, Treatment(reference="L"))[T.P]:C(site)[T.2] = C(trtname, Treatment(reference="L"))[T.C]:C(site)[T.2] = C(trtname, Treatment(reference="L"))[T.M]:C(site)[T.3] = C(trtname, Treatment(reference="L"))[T.P]:C(site)[T.3] = C(trtname, Treatment(reference="L"))[T.C]:C(site)[T.3] = C(trtname, Treatment(reference="L"))[T.M]:C(site)[T.4] = C(trtname, Treatment(reference="L"))[T.P]:C(site)[T.4] = C(trtname, Treatment(reference="L"))[T.C]:C(site)[T.4] = C(trtname, Treatment(reference="L"))[T.M]:C(site)[T.5] = C(trtname, Treatment(reference="L"))[T.P]:C(site)[T.5] = C(trtname, Treatment(reference="L"))[T.C]:C(site)[T.5] = C(trtname, Treatment(reference="L"))[T.M]:C(site)[T.6] = C(trtname, Treatment(reference="L"))[T.P]:C(site

In [322]:
for i, var in enumerate(snap_vars):
    for res in result_snap[i]:
        print(var, rr.f_test_interactions(res, hyps_interactions, alpha))

snainatx   Description     Significance    F-Value       P-Value
0  time_treat    *Significant*  16.061269  2.349415e-10
1  site_treat  Not Significant   1.474889  1.054912e-01
2        site  Not Significant   0.285858  9.210400e-01
snainatx   Description     Significance    F-Value       P-Value
0  time_treat    *Significant*  11.422714  1.850028e-07
1  site_treat  Not Significant   0.778523  7.031027e-01
2        site  Not Significant   0.478532  7.925425e-01
snahypax   Description     Significance    F-Value       P-Value
0  time_treat    *Significant*  21.565982  8.193047e-14
1  site_treat  Not Significant   1.527963  8.670372e-02
2        site  Not Significant   1.069212  3.753642e-01
snahypax   Description     Significance    F-Value       P-Value
0  time_treat    *Significant*  14.544289  1.998723e-09
1  site_treat  Not Significant   1.643687  5.523849e-02
2        site  Not Significant   0.111711  9.898156e-01
snaoddx   Description     Significance   F-Value   P-Value
0  time_t

In [323]:
for i, var in enumerate(ssrs_vars):
    for res in result_ssrs[i]:
        print(var, rr.f_test_interactions(res, hyps_interactions, alpha))

sspintx   Description     Significance   F-Value   P-Value
0  time_treat  Not Significant  0.100833  0.959549
1  site_treat  Not Significant  0.456635  0.961461
2        site  Not Significant  0.368243  0.870591
sspintx   Description     Significance   F-Value   P-Value
0  time_treat  Not Significant  1.776329  0.149427
1  site_treat  Not Significant  0.771165  0.711268
2        site  Not Significant  0.631833  0.675473
ssptossx   Description     Significance   F-Value   P-Value
0  time_treat    *Significant*  5.101886  0.001608
1  site_treat  Not Significant  0.496734  0.943616
2        site  Not Significant  0.485249  0.787516
ssptossx   Description     Significance   F-Value   P-Value
0  time_treat  Not Significant  0.861553  0.460281
1  site_treat  Not Significant  0.935707  0.522976
2        site  Not Significant  0.970836  0.434083


In [351]:
alpha

0.05

In [324]:
f_result_snap  =[]
for i, var in enumerate(snap_vars):
    for res in result_snap[i]:
        f_result_snap.append([var, rr.f_test_interactions(res, hyps_interactions, alpha)])

In [325]:
f_result_ssrs  =[]
for i, var in enumerate(ssrs_vars):
    for res in result_ssrs[i]:
        f_result_ssrs.append([var, rr.f_test_interactions(res, hyps_interactions, alpha)])

[['snainatx',
    Description     Significance    F-Value       P-Value
  0  time_treat    *Significant*  11.422714  1.850028e-07
  1  site_treat  Not Significant   0.778523  7.031027e-01
  2        site  Not Significant   0.478532  7.925425e-01],
 ['snainatx',
    Description     Significance    F-Value       P-Value
  0  time_treat    *Significant*  16.061269  2.349415e-10
  1  site_treat  Not Significant   1.474889  1.054912e-01
  2        site  Not Significant   0.285858  9.210400e-01],
 ['snahypax',
    Description     Significance    F-Value       P-Value
  0  time_treat    *Significant*  14.544289  1.998723e-09
  1  site_treat  Not Significant   1.643687  5.523849e-02
  2        site  Not Significant   0.111711  9.898156e-01],
 ['snahypax',
    Description     Significance    F-Value       P-Value
  0  time_treat    *Significant*  21.565982  8.193047e-14
  1  site_treat  Not Significant   1.527963  8.670372e-02
  2        site  Not Significant   1.069212  3.753642e-01],
 ['snaod

In [336]:
# orig_rr_14 = orig_rr_14[orig_rr_14['Measure and Rater'] != 'Classroom observer' ]
# orig_rr_14 = orig_rr_14[orig_rr_14['Measure and Rater'] != 'MASC Child' ]

# orig_rr_14 = orig_rr_14.reset_index()

In [337]:
# orig_rr_14['replicated F_value'] = None
# orig_rr_14['replicated p_value'] = None
# orig_rr_14['variable'] = None
# k = 0
# for i, res in enumerate(f_result_snap):
#         for j in range(len(res[1])):
#                 print(k, res[0], res[1]['F-Value'][j].round(2))
#                 orig_rr_14.loc[k, 'replicated F_value'] = res[1]['F-Value'][j].round(2)
#                 orig_rr_14.loc[k, 'replicated p_value'] = res[1]['P-Value'][j].round(2)
#                 orig_rr_14.loc[k, 'variable'] = res[0]
#                 k += 1

0 snainatx 16.06
1 snainatx 1.47
2 snainatx 0.29
3 snainatx 11.42
4 snainatx 0.78
5 snainatx 0.48
6 snahypax 21.57
7 snahypax 1.53
8 snahypax 1.07
9 snahypax 14.54
10 snahypax 1.64
11 snahypax 0.11
12 snaoddx 7.17
13 snaoddx 1.16
14 snaoddx 0.72
15 snaoddx 3.42
16 snaoddx 1.75
17 snaoddx 0.4


In [338]:
f_result_ssrs

[['sspintx',
    Description     Significance   F-Value   P-Value
  0  time_treat  Not Significant  0.100833  0.959549
  1  site_treat  Not Significant  0.456635  0.961461
  2        site  Not Significant  0.368243  0.870591],
 ['sspintx',
    Description     Significance   F-Value   P-Value
  0  time_treat  Not Significant  1.776329  0.149427
  1  site_treat  Not Significant  0.771165  0.711268
  2        site  Not Significant  0.631833  0.675473],
 ['ssptossx',
    Description     Significance   F-Value   P-Value
  0  time_treat    *Significant*  5.101886  0.001608
  1  site_treat  Not Significant  0.496734  0.943616
  2        site  Not Significant  0.485249  0.787516],
 ['ssptossx',
    Description     Significance   F-Value   P-Value
  0  time_treat  Not Significant  0.861553  0.460281
  1  site_treat  Not Significant  0.935707  0.522976
  2        site  Not Significant  0.970836  0.434083]]

In [339]:
# k = 18
# for i, res in enumerate(f_result_ssrs):
#         for j in range(len(res[1])):
#                 print(k, res[0], res[1]['F-Value'][j].round(2))
#                 orig_rr_14.loc[k, 'replicated F_value'] = res[1]['F-Value'][j].round(2)
#                 orig_rr_14.loc[k, 'replicated p_value'] = res[1]['P-Value'][j].round(2)
#                 orig_rr_14.loc[k, 'variable'] = res[0]
#                 k += 1

18 sspintx 0.1
19 sspintx 0.46
20 sspintx 0.37
21 sspintx 1.78
22 sspintx 0.77
23 sspintx 0.63
24 ssptossx 5.1
25 ssptossx 0.5
26 ssptossx 0.49
27 ssptossx 0.86
28 ssptossx 0.94
29 ssptossx 0.97


In [363]:
orig_rr_14['replicated p_value'] = orig_rr_14['replicated p_value'].astype(float)

orig_rr_14.style.apply(
    lambda x: ['background-color: blue' if v < alpha else '' for v in x], 
    subset=['replicated p_value'])

,index,Unnamed: 0,Outcome Domain,Measure and Rater,Test,F_value,P_value,replicated F_value,replicated p_value,variable
0,0,0,ADHD symptoms,Inattention - Teacher,Treatment x time,10.600000,<.001,16.060000,0.000000,snainatx
1,1,1,ADHD symptoms,Inattention - Teacher,Treatment x site,0.900000,.56,1.470000,0.110000,snainatx
2,2,2,ADHD symptoms,Inattention - Teacher,Site,2.700000,.02,0.290000,0.920000,snainatx
3,3,3,ADHD symptoms,Inattention - Parent,Treatment x time,21.500000,<.001,11.420000,0.000000,snainatx
4,4,4,ADHD symptoms,Inattention - Parent,Treatment x site,0.600000,.88,0.780000,0.700000,snainatx
5,5,5,ADHD symptoms,Inattention - Parent,Site,3.000000,.02,0.480000,0.790000,snainatx
6,6,6,ADHD symptoms,Hyperactive-impulsive - Teacher,Treatment x time,10.000000,<.001,21.570000,0.000000,snahypax
7,7,7,ADHD symptoms,Hyperactive-impulsive - Teacher,Treatment x site,1.300000,.49,1.530000,0.090000,snahypax
8,8,8,ADHD symptoms,Hyperactive-impulsive - Teacher,Site,3.000000,.02,1.070000,0.380000,snahypax
9,9,9,ADHD symptoms,Hyperactive-impulsive - Parent,Treatment x time,21.500000,<.001,14.540000,0.000000,snahypax


In [341]:
# mean_dict = {}
# std_dict = {}
# for qst, dat in zip( data.keys(), data.values()):
#     dates_meandict = {}
#     dates_stddict = {}
#     for date in ['b', '14', '24', '36']:
#         group_mean= []
#         group_std= []

#         for var in outcomes_dict[qst]:
#             mean, std  = [], []

#             for group in ['C', 'M', 'P']:
            
#                 dat_ = dat[date][dat[date]['trtname'] == group] #extract data from qst[date[var], group 

#                 mean.append(dat_[var].mean())
#                 std.append(dat_[var].std())
#             group_mean.append(mean)
#             group_std.append(std)
#         dates_meandict[date] = group_mean
#         dates_stddict[date] = group_std

            
#     mean_dict[qst] = dates_meandict
#     std_dict[qst] = dates_stddict



In [342]:
# orig_rr_14.to_csv(Path(derived_data, 'original_and_new_rr_14.csv'))

In [350]:
for i, res in enumerate(f_result_ssrs):
    print(res)

['sspintx',   Description     Significance   F-Value   P-Value
0  time_treat  Not Significant  0.100833  0.959549
1  site_treat  Not Significant  0.456635  0.961461
2        site  Not Significant  0.368243  0.870591]
['sspintx',   Description     Significance   F-Value   P-Value
0  time_treat  Not Significant  1.776329  0.149427
1  site_treat  Not Significant  0.771165  0.711268
2        site  Not Significant  0.631833  0.675473]
['ssptossx',   Description     Significance   F-Value   P-Value
0  time_treat    *Significant*  5.101886  0.001608
1  site_treat  Not Significant  0.496734  0.943616
2        site  Not Significant  0.485249  0.787516]
['ssptossx',   Description     Significance   F-Value   P-Value
0  time_treat  Not Significant  0.861553  0.460281
1  site_treat  Not Significant  0.935707  0.522976
2        site  Not Significant  0.970836  0.434083]


- all snap vars are significant 
- ssrs 'sspintx', parent  not significant 



,0,1
0,snainatx,Description Significance F-Value ...
1,snainatx,Description Significance F-Value ...
2,snahypax,Description Significance F-Value ...
3,snahypax,Description Significance F-Value ...
4,snaoddx,Description Significance F-Value P-V...
5,snaoddx,Description Significance F-Value P-V...


## contrasts 